In [ ]:
!pip install trl

In [ ]:
!huggingface-cli login
!wandb login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModelForCausalLM
from datasets import load_dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
).to("cuda:0")

# Carrega o dataset já formatado
dataset = load_dataset("verolfelipe/drug_bank_metabolism_absorption", split="train")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

# FORMATA DATASET PARA PADRÃO MISTRAL
dataset_format = dataset.map(lambda x: {"text": tokenizer.apply_chat_template(x["messages"], tokenize=False, add_generation_prompt=False)})

peft_config = LoraConfig(
    r=8,  # Rank dimension - typically between 4-32
    lora_alpha=16,  # LoRA scaling factor - typically 2x rank
    lora_dropout=0.05,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

# Configura o treinamento
training_args = SFTConfig(
    output_dir="./finetuned_models",
    max_steps=1000,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    gradient_checkpointing=True,
    fp16=True,
)

# Treina o modelo
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_format,
    peft_config=peft_config,
    processing_class=tokenizer
)

trainer.train()

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/4697 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/4697 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/4697 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: felipeverol (felipeverol-cti-renato-archer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.485000
20,2.072100
30,1.594500
40,1.507800
50,1.605100
60,1.430000
70,1.471900
80,1.354800
90,1.562100
100,1.433800


TrainOutput(global_step=1000, training_loss=1.357377477645874, metrics={'train_runtime': 4244.0853, 'train_samples_per_second': 0.942, 'train_steps_per_second': 0.236, 'total_flos': 4.176224907961958e+16, 'train_loss': 1.357377477645874})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModel
from datasets import load_dataset
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

peft_model = PeftModel.from_pretrained(
    model, "/content/finetuned_models/checkpoint-1000/", torch_dtype=torch.float16
)

# 3. Merge adapter weights with base model
merged_model = peft_model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
!zip -r mistral-lora-checkpoint.zip /content/finetuned_models


  adding: content/finetuned_models/ (stored 0%)
  adding: content/finetuned_models/checkpoint-700/ (stored 0%)
  adding: content/finetuned_models/checkpoint-700/scaler.pt (deflated 60%)
  adding: content/finetuned_models/checkpoint-700/rng_state.pth (deflated 25%)
  adding: content/finetuned_models/checkpoint-700/optimizer.pt (deflated 9%)
  adding: content/finetuned_models/checkpoint-700/tokenizer.model (deflated 61%)
  adding: content/finetuned_models/checkpoint-700/adapter_model.safetensors (deflated 7%)
  adding: content/finetuned_models/checkpoint-700/tokenizer_config.json (deflated 95%)
  adding: content/finetuned_models/checkpoint-700/adapter_config.json (deflated 56%)
  adding: content/finetuned_models/checkpoint-700/scheduler.pt (deflated 55%)
  adding: content/finetuned_models/checkpoint-700/tokenizer.json (deflated 85%)
  adding: content/finetuned_models/checkpoint-700/README.md (deflated 66%)
  adding: content/finetuned_models/checkpoint-700/trainer_state.json (deflated 80%

In [ ]:
!huggingface-cli login
!wandb login

merged_model.push_to_hub("verolfelipe/Mistral-Metabolism_Absorption")
tokenizer.push_to_hub("verolfelipe/Mistral-Metabolism_Absorption")


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/verolfelipe/Mistral-Metabolism_Absorption/commit/37246bdc8bf0f5428bf26d0d53aeaec61998e3e5', commit_message='Upload tokenizer', commit_description='', oid='37246bdc8bf0f5428bf26d0d53aeaec61998e3e5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/verolfelipe/Mistral-Metabolism_Absorption', endpoint='https://huggingface.co', repo_type='model', repo_id='verolfelipe/Mistral-Metabolism_Absorption'), pr_revision=None, pr_num=None)

In [ ]:
!huggingface-cli login
!wandb login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModel
from datasets import load_dataset
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

lora_model = PeftModel.from_pretrained(
    model, "/content/finetuned_models/checkpoint-1000/", torch_dtype=torch.float16
)

lora_model.push_to_hub("verolfelipe/Mistral-Metabolism-Absorption-LoRA", commit_message="Upload LoRA checkpoint sem merge")
tokenizer.push_to_hub("verolfelipe/Mistral-Metabolism-Absorption-LoRA")
model_merged = lora_model.merge_and_unload()
model_merged.push_to_hub("verolfelipe/Mistral-Metabolism-Absorption", commit_message="Upload merged model")
tokenizer.push_to_hub("verolfelipe/Mistral-Metabolism-Absorption")


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/verolfelipe/Mistral-Metabolism-Absorption/commit/47140a58de1fa4dabc1a1d4d1666bb0855a87a81', commit_message='Upload tokenizer', commit_description='', oid='47140a58de1fa4dabc1a1d4d1666bb0855a87a81', pr_url=None, repo_url=RepoUrl('https://huggingface.co/verolfelipe/Mistral-Metabolism-Absorption', endpoint='https://huggingface.co', repo_type='model', repo_id='verolfelipe/Mistral-Metabolism-Absorption'), pr_revision=None, pr_num=None)

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import pipeline

pipe = pipeline("text-generation", model="verolfelipe/Mistral-Metabolism-Absorption", tokenizer="verolfelipe/Mistral-Metabolism-Absorption")

def respond(message, chat_history):
    chat_history = chat_history or []
    response = pipe(message, max_new_tokens=100, do_sample=True, temperature=0.7)[0]['generated_text']
    chat_history.append((message, response))
    return chat_history, chat_history

iface = gr.Interface(
    fn=respond,
    inputs=["text", "state"],
    outputs=["chatbot", "state"],
    title="Chat com Meu Modelo",
)

iface.launch()

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/gradio/components/base.py:433: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  obj = utils.component_or_layout_class(cls_name)(render=render)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbb71ab071f98b852b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
